In [1]:
import pandas as pd
from statsbombpy import sb

import requests
import json
import http.client
import json
import math
import numpy as np

from bs4 import BeautifulSoup

Competitions - sb.competitions()
Matches - sb.matches(competition_id=9, season_id=42)
Lineups - sb.lineups(match_id=303299)
Events - sb.events(match_id=303299)

Competition Events - events = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2019/2020",
    gender="male"
)

grouped_events = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2019/2020",
    split=True
)
grouped_events["dribbles"]

360 data
match_frames = sb.frames(match_id=3895302, fmt='dataframe')
comp_frames = sb.competition_frames(
    country="Germany",
    division= "1. Bundesliga",
    season="2023/2024"
)

In [ ]:
team = 'Bayer Leverkusen'

# Get all the match_ids
matches_df = sb.matches(competition_id=9, season_id=281)
match_list = matches_df['match_id'].to_list()

# Define a finctuoin the uses the scores to determine a win, draw or lose
def opponent(row):
    
    if row['home_team'] == team:
        return row['away_team']
    elif row['away_team'] == team:
        return row['home_team']
    else:
        return None
    
matches_df['opponent'] = matches_df.apply(opponent, axis = 1)
matches_df.head(10)
# Export players info
#matches_df.to_csv(r'./MatchInfo.csv', index=False)

In [4]:
matches_df[matches_df['match_id'] == 3895333]

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version,opponent
2,3895333,2024-05-05,18:30:00.000,Germany - 1. Bundesliga,2023/2024,Eintracht Frankfurt,Bayer Leverkusen,1,5,available,...,32,Regular Season,Deutsche Bank Park,Christian Dingert,Dino Toppmöller,Xabier Alonso Olano,1.1.0,2,2,Eintracht Frankfurt


In [ ]:
# Get all player_id, player_name, jersey_number and country for all fixtures
players_list = []

for i in matches_df['match_id']:
    lineup = sb.lineups(match_id = i)[team]
    fullLineup = lineup[['player_id','player_name','jersey_number','country']]
    players_list.append(fullLineup)

players_df = pd.concat(players_list, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)

# Export players info
# players_df.to_csv(r'./PlayersList.csv', index=False)

In [ ]:
#Store event data
events_list = []

for i in matches_df['match_id']:
    m_events = sb.events(i)
    m_events['match_id'] = i # Column to store the match_id
    events_list.append(m_events)

# Join the dataframes into a single one
events_df = pd.concat(events_list, axis = 0, ignore_index=True)

# Split columns
events_df[['location_x', 'location_y']] = events_df['location'].apply(pd.Series)
events_df[['pass_end_location_x', 'pass_end_location_y']] = events_df['pass_end_location'].apply(pd.Series)
events_df['shot_end_location'] = events_df['shot_end_location'].astype(str)
events_df[['shot_end_location_x', 'shot_end_location_y', 'shot_end_location_z']] = events_df['shot_end_location'].str.strip('[]').str.split(', ', expand=True)

# Passing

In [26]:
# Get Leverkusen Passes
allpass_df = events_df[(events_df['type'] == 'Pass') & (events_df['team'] == team)]
#completePass_df = allpass_df[allpass_df['pass_outcome'].isna() == True]

# Filter needed data
passNetworkDF = allpass_df[['id','match_id','period','minute','player','player_id','position',
                'location_x','location_y','pass_recipient','pass_recipient_id',
                'pass_end_location_x','pass_end_location_y','pass_type','pass_technique',
                'pass_height','pass_length','pass_outcome','shot_outcome','pass_goal_assist']]

def passoutcome(row):
    if pd.isna(row['pass_outcome']):
        return 'complete'
    elif row['pass_outcome'] == 'Incomplete':
        return 'incomplete'
    else:
        return None
    

# Applky the function and create a new column
passNetworkDF['pass_success'] = passNetworkDF.apply(passoutcome, axis = 1)

# Reset index
passNetworkDF.reset_index(drop=True, inplace=True)

# # Assign assists
for i in range(len(passNetworkDF)):  # Start from 1 to avoid index error
    if (passNetworkDF.loc[i, 'pass_goal_assist'] == True):
        # Replace pass success to the assist
        passNetworkDF.at[i, 'pass_success'] = 'GoalAssist'

# Filter pass outcome for only incomplete and nan/complete passes
fullpassNetworkDF = passNetworkDF[(passNetworkDF['pass_outcome'].isna() == True) | (passNetworkDF['pass_outcome'] == 'Incomplete')]

# if 'pass_outcome' isna == True then assign complete
# is != True then assign Nan value

fullpassmap = fullpassNetworkDF.reset_index(drop=True)
fullpassmap['player_id'] = fullpassmap['player_id'].astype(int)
#fullpassmap.to_csv(r'./FullPassMap.csv', index=False)

C:\Users\Razak\AppData\Local\Temp\ipykernel_12880\1536929201.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passNetworkDF['pass_success'] = passNetworkDF.apply(passoutcome, axis = 1)


In [24]:
allpass_df[allpass_df['pass_goal_assist'] == True]['player'].value_counts()

player
Alejandro Grimaldo García         13
Florian Wirtz                     10
Victor Okoh Boniface               8
Jonas Hofmann                      7
Jeremie Frimpong                   7
Amine Adli                         5
Exequiel Alejandro Palacios        4
Robert Andrich                     2
Adam Hložek                        2
Arthur Augusto de Matos Soares     2
Nathan Tella                       2
Josip Stanišić                     1
Jonathan Tah                       1
Edmond Fayçal Tapsoba              1
Piero Martín Hincapié Reyna        1
Noah Mbamba                        1
Name: count, dtype: int64

In [17]:
allpass_df[allpass_df['pass_technique'] == 'Through Ball']['player'].value_counts()

player
Florian Wirtz                  22
Granit Xhaka                   19
Alejandro Grimaldo García      15
Exequiel Alejandro Palacios     9
Victor Okoh Boniface            7
Edmond Fayçal Tapsoba           7
Jeremie Frimpong                7
Odilon Kossonou                 6
Robert Andrich                  5
Jonas Hofmann                   5
Amine Adli                      4
Piero Martín Hincapié Reyna     3
Josip Stanišić                  3
Nathan Tella                    2
Patrik Schick                   1
Name: count, dtype: int64

In [16]:
allpass_df[allpass_df['pass_cross'] == True]['player'].value_counts()

player
Alejandro Grimaldo García         106
Jeremie Frimpong                   95
Jonas Hofmann                      52
Florian Wirtz                      50
Nathan Tella                       27
Amine Adli                         27
Josip Stanišić                     12
Robert Andrich                     11
Victor Okoh Boniface               11
Arthur Augusto de Matos Soares      9
Granit Xhaka                        9
Piero Martín Hincapié Reyna         7
Exequiel Alejandro Palacios         6
Edmond Fayçal Tapsoba               5
Jonathan Tah                        4
Adam Hložek                         2
Patrik Schick                       2
Odilon Kossonou                     1
Borja Iglesias Quintas              1
Noah Mbamba                         1
Name: count, dtype: int64

### GameWeek Pass Network

In [11]:
# Get completed passes
completePasses = fullpassNetworkDF[fullpassNetworkDF['pass_success'] == 'complete']

avgLocationperGame = completePasses.groupby(['player','match_id']).agg({'location_x': ['mean'], 'location_y':['mean','count']})
avgLocationperGame.columns = ['location_x','location_y','count']

In [12]:
passBetween = completePasses.groupby(['player','pass_recipient','match_id']).id.count().reset_index()
passBetween.rename({'id':'pass_count'}, axis='columns', inplace=True)

In [13]:
passBetween = passBetween.merge(avgLocationperGame, left_on = ['player','match_id'], right_index=True)
passBetween = passBetween.merge(avgLocationperGame, left_on= ['pass_recipient','match_id'], right_index=True, suffixes=['','_end'])

In [14]:
#meanCount = passBetween['pass_count'].mean()
passBetween = passBetween[passBetween['pass_count'] > 1]
#passBetween.to_csv(r'./FullPassNetwork.csv', index=False)

In [15]:
rwRecords = passBetween[['match_id']].drop_duplicates().sort_values(by='match_id').reset_index(drop=True)
rwRecords = pd.DataFrame(rwRecords)
rwRecords['matchWeek'] = rwRecords.index + 1

rwRecords.index = rwRecords.index.astype(int)

rwRecords['Column'] = rwRecords.index % 5
rwRecords['Row'] = (rwRecords.index // 5) % 7

rwRecords.reset_index(drop=True, inplace=True)
#rwRecords.to_csv(r'./PassNetworkSmallMultiples.csv', index=False)

### Average Pass Networkn (Season)

In [16]:
avgLocationperGame = completePasses.groupby(['player']).agg({'location_x': ['mean'], 'location_y':['mean','count']})
avgLocationperGame.columns = ['location_x','location_y','count']
avgLocationperGame.head()

,location_x,location_y,count
player,,,
Adam Hložek,71.939535,37.231008,129
Alejandro Grimaldo García,70.465198,14.052316,1770
Amine Adli,77.263144,27.126829,369
Arthur Augusto de Matos Soares,70.610870,25.894565,92
Borja Iglesias Quintas,84.240000,34.005000,60


In [17]:
passBetween = completePasses.groupby(['player','pass_recipient']).id.count().reset_index()
passBetween.rename({'id':'pass_count'}, axis='columns', inplace=True)
passBetween.head()

,player,pass_recipient,pass_count
0,Adam Hložek,Alejandro Grimaldo García,17
1,Adam Hložek,Amine Adli,4
2,Adam Hložek,Arthur Augusto de Matos Soares,6
3,Adam Hložek,Borja Iglesias Quintas,2
4,Adam Hložek,Edmond Fayçal Tapsoba,2


In [18]:
passBetween = passBetween.merge(avgLocationperGame, left_on = ['player'], right_index=True)
passBetween = passBetween.merge(avgLocationperGame, left_on= ['pass_recipient'], right_index=True, suffixes=['','_end'])

In [19]:
passBetween = passBetween[passBetween['pass_count'] > 55]

In [20]:
#passBetween.to_csv(r'./SeasonPassNetwork.csv', index=False)

### Passmap

In [22]:
passMap = events_df[events_df['team'] == team]

# Filter needed data
passMapDF = passMap[['id','match_id','period','minute','player','player_id','position',
                'location_x','location_y','pass_recipient','pass_recipient_id',
                'pass_end_location_x','pass_end_location_y','pass_type',
                'pass_height','pass_length','pass_outcome','shot_outcome','pass_goal_assist']]

def passoutcome(row):
    if pd.isna(row['pass_outcome']):
        return 'complete'
    elif row['pass_outcome'] == 'Incomplete':
        return 'incomplete'
    else:
        return None
    
# Apply the function and create a new column
passMapDF['pass_success'] = passMapDF.apply(passoutcome, axis = 1)

# Reset index
passMapDF.reset_index(drop=True, inplace=True)

# # Assign assists
for i in range(len(passMapDF)):  # Start from 1 to avoid index error
    if (passMapDF.loc[i, 'pass_goal_assist'] == True):
        # Replace pass success to the assist
        passMapDF.at[i, 'pass_success'] = 'GoalAssist'

#passMapDF.head()

C:\Users\Razak\AppData\Local\Temp\ipykernel_11356\1965100703.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passMapDF['pass_success'] = passMapDF.apply(passoutcome, axis = 1)


# Average Position

In [23]:
leverkusenDF = events_df[events_df['team'] == team]
a = leverkusenDF[['player','player_id','position','location_x','location_y','minute']]

b = a.groupby(['position']).agg({'location_x': ['mean'], 'location_y':['mean','count']})
b.columns = ['_'.join(col).strip() if col[1] else col[0] for col in b.columns.values]
c = b.reset_index()
#c['player_id'] = c['player_id'].astype(int)
dd = c.sort_values('location_y_count', ascending=False)
#dd.to_csv(r'./AveragePositionbyName.csv', index=False)

In [24]:
b = a.groupby(['player','player_id']).agg({'location_x': ['mean'], 'location_y':['mean','count']})
b.columns = ['_'.join(col).strip() if col[1] else col[0] for col in b.columns.values]
c = b.reset_index()
c['player_id'] = c['player_id'].astype(int)
d = c.sort_values('location_y_count', ascending=False)
#d.to_csv(r'./AveragePosition.csv', index=False)

# xG Plot

In [6]:
xg_df = events_df[events_df['type'] == 'Shot']
a = xg_df[['match_id','team','shot_statsbomb_xg','shot_outcome']]
b = a.groupby(['match_id','team']).sum('shot_statsbomb_xg').reset_index()
c = b.sort_values('match_id')


def xflag(row):
    if row['team'] == team:
        return 'xG Created'
    elif row['team'] != team:
        return 'xG Conceded'
    else:
        return None

c['xFlag'] = c.apply(xflag, axis = 1)
#c.to_csv(r'./xGValues.csv', index =False)

In [27]:
c.groupby('xFlag').sum()['shot_statsbomb_xg']

xFlag
xG Conceded    27.807305
xG Created     72.695292
Name: shot_statsbomb_xg, dtype: float64

# League Table

In [28]:
# For loop that scrapes 
for gameweek in range(1,35):
    url = f'https://www.transfermarkt.com/bundesliga/spieltagtabelle/wettbewerb/L1?saison_id=2023&spieltag={gameweek}'
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    ab = soup.select('div[class="box"] table[class="items"] tbody')[0]
    c = ab.text.split('\n')
    d = [item for item in c if item.strip()]
    print('Gameweek ' + str(gameweek))
    for i in range(0, len(d), 3):
        joined = d[i].strip() + ' ' + d[i+1].strip()
        print(joined)    

In [29]:
lg_standing = pd.read_csv(r'./League Position.csv')
lg_standing.head()


In [30]:
lg_standing = lg_standing.iloc[:,1:]
lg_standing = lg_standing.T #Transpose
lg_standing.columns = lg_standing.iloc[0]
lg_standing = lg_standing.iloc[1:]
lg_standing.head()

In [31]:
#lg_standing.to_csv(r'./LeagueStanding.csv', index=True)

# Shotmap / Goal location

In [32]:
events_df['shot_outcome'].unique()

array([nan, 'Blocked', 'Saved', 'Wayward', 'Goal', 'Post', 'Off T',
       'Saved Off Target', 'Saved to Post'], dtype=object)

In [33]:
goalDf = events_df[events_df['shot_outcome'] == 'Goal']

In [7]:
shotmapDf = xg_df[['id','match_id','minute','player','player_id','shot_body_part',
        'shot_outcome', 'shot_statsbomb_xg','location_x', 'location_y','shot_end_location_x',
        'shot_end_location_y', 'shot_end_location_z','team', 'team_id']].reset_index(drop=True)
shotmapDf['player_id'] = shotmapDf['player_id'].astype(int)


In [8]:
shotmapDf.loc[shotmapDf['team'] != team, 'location_x'] = 120 - shotmapDf['location_x']
#shotmapDf.loc[shotmapDf['team'] != team, 'location_y'] = 80 - shotmapDf['location_y']

shotmapDf.loc[shotmapDf['team'] != team, 'shot_end_location_x'] = 120 - shotmapDf['shot_end_location_x'].astype(float)
#shotmapDf.loc[shotmapDf['team'] != team, 'shot_end_location_y'] = 80 - shotmapDf['shot_end_location_y'].astype(float)

#shotmapDf.to_csv(r'./Shotmap.csv', index=False)

In [37]:
rwRecords = shotmapDf[['match_id']].drop_duplicates().sort_values(by='match_id').reset_index(drop=True)
rwRecords = pd.DataFrame(rwRecords)
rwRecords['matchWeek'] = rwRecords.index + 1

rwRecords.index = rwRecords.index.astype(int)

rwRecords['Column'] = rwRecords.index % 4
rwRecords['Row'] = (rwRecords.index // 4) % 9

rwRecords.reset_index(drop=True, inplace=True)
#rwRecords.to_csv(r'./GoalSmallMultiples.csv', index=False)


### Player Goal Involvement

In [40]:
goalCont = leverkusenDF[(leverkusenDF['shot_outcome'] == 'Goal') | (leverkusenDF['pass_goal_assist'] == True)]
goalCntDF = goalCont[['id','match_id','minute','player','player_id','shot_body_part',
        'shot_outcome', 'shot_statsbomb_xg','location_x', 'location_y','shot_end_location_x',
        'shot_end_location_y', 'shot_end_location_z','pass_goal_assist','pass_end_location_x','pass_end_location_y']].reset_index(drop=True)
goalCntDF['player_id'] = goalCntDF['player_id'].astype(int)
goalCntDF['goal_outcome'] = goalCntDF.apply(
    lambda row: row['shot_outcome'] if pd.notna(row['shot_outcome']) else ('Assist' if row['pass_goal_assist'] else None),
    axis=1
)
def shotdistance(row):
    if row['goal_outcome'] == 'Goal':
        # Store the location
        x_start = float(row['location_x'])
        y_start = float(row['location_y'])
        z_start = 0.0

        start_location = np.array([x_start, y_start, z_start])

        # End location
        x_end = float(row['shot_end_location_x'])
        y_end = float(row['shot_end_location_y'])
        z_end = float(row['shot_end_location_z'])

        end_location = np.array([x_end, y_end, z_end])

        # Calculate the distance
        sq_distance = np.sum((start_location - end_location) ** 2, axis=0)
        distance = np.sqrt(sq_distance)
        return distance
    else:
        return None

goalCntDF['distance'] = goalCntDF.apply(shotdistance, axis=1)
#goalCntDF.to_csv(r'./GoalContribution.csv', index=False)

## Cumulative Dashboard

In [41]:
leverkusenDF = events_df[events_df['team'] == team]
goalFor = leverkusenDF[leverkusenDF['shot_outcome'] == 'Goal']
gf = goalFor.groupby('minute').count()['id'].reset_index(name='count')
#gf.to_csv(r'./GoalForMinutes.csv', index=False)

opponent = events_df[events_df['team'] != team]
goalAgainst = opponent[opponent['shot_outcome'] == 'Goal']
ga = goalAgainst.groupby('minute').count()['id'].reset_index(name='count')
#ga.to_csv(r'./GoalAgainstMinutes.csv', index=False)

In [ ]:
goal = leverkusenDF[leverkusenDF['shot_outcome'] == 'Goal']
goal.shape

(86, 116)

### Shot Graph, Stats, xG Plot (For and Against)

In [ ]:
a = events_df[events_df['type'] == 'Shot']
sg = a[['id','match_id','minute','player','player_id','shot_body_part',
        'shot_outcome', 'shot_statsbomb_xg','pass_goal_assist','team','team_id']].reset_index(drop=True)
#sg.to_csv(r'./FullShotmap.csv', index=False)

In [ ]:
xg_df = events_df
a = xg_df[['match_id','team','shot_statsbomb_xg','shot_outcome']]
b = a.groupby(['match_id','team']).sum('shot_statsbomb_xg').reset_index()
c = b.sort_values('match_id')


def xflag(row):
    if row['team'] == team:
        return 'xG Created'
    elif row['team'] != team:
        return 'xG Conceded'
    else:
        return None

c['xFlag'] = c.apply(xflag, axis = 1)
c

,match_id,team,shot_statsbomb_xg,xFlag
0,3895052,Bayer Leverkusen,1.523364,xG Created
1,3895052,RB Leipzig,1.722110,xG Conceded
2,3895060,Bayer Leverkusen,2.719871,xG Created
3,3895060,Borussia Mönchengladbach,0.799265,xG Conceded
4,3895067,Bayer Leverkusen,3.304641,xG Created
...,...,...,...,...
63,3895333,Eintracht Frankfurt,1.441534,xG Conceded
64,3895340,Bayer Leverkusen,4.301661,xG Created
65,3895340,Bochum,0.314100,xG Conceded
66,3895348,Augsburg,0.780332,xG Conceded
